In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import math
import csv

In [14]:
with open('titleclaim.csv','r') as title:
    head = csv.reader(title)
with open('bodyclaim.csv','r') as text:
    body = csv.reader(text) 

FileNotFoundError: [Errno 2] No such file or directory: 'titleclaim.csv'

In [15]:
headline = "Two Altar Boys Arrested for Putting Weed in the Censer Burner"
body = """What started as a joke ended with the future of two altar boys from Spain. They were detained overnight, after having surprised them putting weed in the censer-burner of the Cathedral of Santiago de Compostela.
The censer-burner is used the Cathedral of Santiago de Compostela to celebrate the Epiphany of the Lord. Several assistants stated that in this occasion the holy precinct was suddenly covered in an odd smell “it did not smell as always, it was a familiar smell but I could not relate it to anything, but in my son’s bedroom sometimes smell like that”.
Following the Mass, these altar boys were arrested by the police after confirming that the strange smell was correspond to marijuana, “it was a joke, the idea came during the Christmas Eve mass, we bought no more than half a kilo of Mary Jane and we drop it inside the censer-burner, we are sure that people has left of the Cathedral happier more than ever”. Finally, they were freed without charge but they will not be able to discharge their functions as altar boys any more."""
diff1 =  """What started as a joke ended with the future of two altar boys from Spain. They were detained overnight, after having surprised them putting weed in the censer-burner of the Cathedral of Santiago de Compostela.
The censer-burner is used the Cathedral of Santiago de Compostela to celebrate the Epiphany of the Lord. Several assistants stated that in this occasion the holy precinct was suddenly covered in an odd smell “it did not smell as always, it was a familiar smell but I could not relate it to anything, but in my son’s bedroom sometimes smell like that”.
Following the Mass, these altar boys were arrested by the police after confirming that the strange smell was correspond to marijuana, “it was a joke, the idea came during the Christmas Eve mass, we bought no more than half a kilo of Mary Jane and we drop it inside the censer-burner, we are sure that people has left of the Cathedral happier more than ever”. Finally, they were freed without charge but they will not be able to discharge their functions as altar boys any more."""
diff2 = """Based on data and analyses prevented by its high-level technical experts committee on coronavirus disease (Covid-19), the health ministry is likely to extend testing protocols to include rapid antibody testing for everyone at risk, including those outside hotspot areasOnce approved, these guidelines will replace the Indian Council of Medical Research’s interim advisory issued on April 2 that limited rapid antibody testing to hotspot areas.Unlike the RT-PCR test that detects the virus’ genetic material (RNA) in throat swabs to diagnose current infection, rapid antibody test indicate if a person has been infected and has immunity.The simple blood test, which takes 15-20 minutes to give results, will help identify people who were infected but never diagnosed, which will help map undetected infection and give the correct extend of the epidemic."""
diff = """An article about two altar boys being arrested for putting marijuana in a censer (incense burner) at the Cathedral of Santiago de Compostela in Spain was recirculated on social media in December 2018. This is not a genuine news report.The story appears to have first gone viral after it was published on the Spanish-language satirical web site haynoticia.es in January 2018. According to a report in the newspaper El Pais, more than 2 million people visited Haynoticia.es to read the story, it racked up more than 250,000 interactions on social media, and at least one genuine newspaper republished the article:"""

In [16]:
bowHead = headline.split(" ")
bowBody = body.split(" ")
#print(bowHead)
#print(bowBody)
wordDictHead = dict.fromkeys(bowHead, 0)
wordDictBody = dict.fromkeys(bowBody, 0)
for word in bowHead:
    wordDictHead[word] +=1
for word in bowBody:
    wordDictBody[word] +=1
print(wordDictHead)
print(wordDictBody)


<class 'str'>
{'Two': 1, 'Altar': 1, 'Boys': 1, 'Arrested': 1, 'for': 1, 'Putting': 1, 'Weed': 1, 'in': 1, 'the': 1, 'Censer': 1, 'Burner': 1}
{'What': 1, 'started': 1, 'as': 3, 'a': 4, 'joke': 1, 'ended': 1, 'with': 1, 'the': 14, 'future': 1, 'of': 7, 'two': 1, 'altar': 3, 'boys': 3, 'from': 1, 'Spain.': 1, 'They': 1, 'were': 3, 'detained': 1, 'overnight,': 1, 'after': 2, 'having': 1, 'surprised': 1, 'them': 1, 'putting': 1, 'weed': 1, 'in': 4, 'censer-burner': 2, 'Cathedral': 3, 'Santiago': 2, 'de': 2, 'Compostela.\nThe': 1, 'is': 1, 'used': 1, 'Compostela': 1, 'to': 4, 'celebrate': 1, 'Epiphany': 1, 'Lord.': 1, 'Several': 1, 'assistants': 1, 'stated': 1, 'that': 3, 'this': 1, 'occasion': 1, 'holy': 1, 'precinct': 1, 'was': 4, 'suddenly': 1, 'covered': 1, 'an': 1, 'odd': 1, 'smell': 5, '“it': 2, 'did': 1, 'not': 3, 'always,': 1, 'it': 3, 'familiar': 1, 'but': 3, 'I': 1, 'could': 1, 'relate': 1, 'anything,': 1, 'my': 1, 'son’s': 1, 'bedroom': 1, 'sometimes': 1, 'like': 1, 'that”.\nFol

In [5]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count /float(bowCount)
    return tfDict

In [6]:
def computeIDF(docList):
    idfDict = {}
    n = len(docList)
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] +=1
    for word, val in idfDict.items():
        idfDict[word] = math.log10(n/float(val))
    return idfDict

In [7]:
def build_vector(iterable1, iterable2):
    counter1 = Counter(iterable1)
    counter2 = Counter(iterable2)
    all_items = set(counter1.keys()).union(set(counter2.keys()))
    vector1 = [counter1[k] for k in all_items]
    vector2 = [counter2[k] for k in all_items]
    return vector1, vector2

def cosim(v1, v2):
    dot_product = sum(n1 * n2 for n1, n2 in zip(v1, v2) )
    magnitude1 = math.sqrt(sum(n ** 2 for n in v1))
    magnitude2 = math.sqrt(sum(n ** 2 for n in v2))
    return dot_product / (magnitude1 * magnitude2)

In [17]:
# This is the tfidf cosine similarity block
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_feature_head = tfidf_vectorizer.fit_transform(bowHead)
tfidf_feature_body = tfidf_vectorizer.fit_transform(bowBody)
document = (headline, body, diff)
tfidf_matrix = tfidf_vectorizer.fit_transform(document)
similar = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)
print(tfidf_matrix.shape)
print(similar)

<class 'tuple'>
(3, 101)
[[1.         0.32097079 0.15244237]]


In [10]:
print(type(similar))

<class 'numpy.ndarray'>


In [12]:
for x in np.nditer(similar):
    print(x)

1.0
0.3209707913039122
0.1524423658968604
